In [17]:
#공통 import
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_validate

from joblib import parallel_backend
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA

#모델 import
from xgboost import XGBRFClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier

from sklearn.metrics import accuracy_score
# df_train = pd.read_csv('./통합데이터/df_train_v3_mean.csv')
# df_train = pd.read_csv('./통합데이터/df_train_v3.csv')
# df_train = pd.read_csv('./통합데이터/df_train_v3_mean_1m.csv')
df_train = pd.read_csv('./df_train_v3_mean_1m_이상치제거.csv')

df_test = pd.read_csv("./통합데이터/df_test_v4_mean.csv")

'''
# 시간 컬럼 제거
df_test_drop = df_test.drop(columns=['year', 'month', 'day', 'hour', 'min'])

# 데이터 전처리
data = df_train.drop(columns=['answer', 'year', 'month', 'day', 'hour'])
target = df_train['answer']
'''

# 입력 데이터, 타깃 데이터, 테스트 데이터 설정
data = df_train[['lat', 'lon', 'sog', 'cog', 'hdg', 'khnp_buoy.ws', 'khnp_buoy.wd', 'kma_lightbecon.ws', 'kma_lightbecon.wd', 'kma_pagobuoy.mean_wh']].to_numpy()
target = df_train['answer'].to_numpy()

test_data = df_test[['lat', 'lon', 'sog', 'cog', 'hdg','khnp_buoy.ws', 'khnp_buoy.wd', 'kma_lightbecon.ws', 'kma_lightbecon.wd', 'kma_pagobuoy.mean_wh']].to_numpy()

# 스케일링
scaler = StandardScaler() #<- 스케일링 방식 변경 가능
data_scaled = scaler.fit_transform(data)
# df_test_scaled = scaler.transform(df_test_drop)
df_test_scaled = scaler.transform(test_data)

'''
# PCA를 사용하여 차원 축소
pca = PCA(n_components = 10)  # 목표 차원 수로 조정
data_pca = pca.fit_transform(data_scaled)

# 축소된 데이터로 새로운 데이터프레임 생성
df_pca = pd.DataFrame(data_pca, columns=['PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6', 'PC7', 'PC8', 'PC9', 'PC10'])
'''

# 축소된 데이터로 모델 훈련 데이터 나누기
# X_train, X_val, y_train, y_val = train_test_split(df_pca, target, test_size=0.2, random_state=123, stratify=target)
X_train, X_val, y_train, y_val = train_test_split(data_scaled, target, test_size=0.2, random_state=123, stratify=target)

# 모델 객체 생성
rfc = RandomForestClassifier(n_estimators=100, max_depth=13, n_jobs=-1, random_state=42) # max_depth 13으로 바꿈
adaboost = AdaBoostClassifier(random_state=42)
xgbrfc = XGBRFClassifier()

# 모델 훈련
rfc.fit(X_train, y_train)
# adaboost(X_train, y_train)
# xgbrfc(X_train, y_train)

# 훈련 세트 정확도 출력
train_pred = rfc.predict(X_train)
train_accuracy = accuracy_score(y_train, train_pred)
print("훈련 세트 정확도:", train_accuracy)

# 검증 세트 정확도 출력
val_pred = rfc.predict(X_val)
val_accuracy = accuracy_score(y_val, val_pred)
print("검증 세트 정확도:", val_accuracy)

# test 데이터 예측
print("test 데이터 예측", rfc.predict(df_test_scaled))

# 예측 결과 dataframe 생성
pd.DataFrame({'year':df_test.year, 'month':df_test.month, 'day':df_test.day, 'hour':df_test.hour, 'min':df_test['min'], 'target':rfc.predict(df_test_scaled)}).to_csv('result(scaled).csv', index=False)

# 예측 결과 확인
result = pd.read_csv('./제출 양식/result(scaled).csv')
print(result)

# 예측 결과 편중 확인
x = rfc.predict(df_test_scaled)
print(np.unique(x, return_counts=True))

# CSI 계산
from sklearn.metrics import confusion_matrix

def calculate_csi(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    csi = tp / (tp + fn + fp) * 100
    return csi

print("훈련 세트에 대한 csi", calculate_csi(y_train, train_pred))
print("검증 세트에 대한 csi", calculate_csi(y_val, val_pred))

훈련 세트 정확도: 0.9907739453608919
검증 세트 정확도: 0.9861340519864383
test 데이터 예측 [1 1 1 ... 0 0 0]
       year  month  day  hour  min  target
0      2022      8   24    14   41       1
1      2022      8   24    14   42       1
2      2022      8   24    14   43       1
3      2022      8   24    14   44       1
4      2022      8   24    14   45       1
...     ...    ...  ...   ...  ...     ...
51132  2023      1   17    19   44       0
51133  2023      1   17    19   50       0
51134  2023      1   17    19   56       0
51135  2023      1   17    19   59       0
51136  2023      1   17    20    5       0

[51137 rows x 6 columns]
(array([0, 1], dtype=int64), array([20694, 30443], dtype=int64))
훈련 세트에 대한 csi 98.52823908747357
검증 세트에 대한 csi 97.79954473339312
